# Stroke Prediction with ensemble
 - Random Forest Classifier : Accuracy - 0.90 
 - Gradient Boosting Classifier : Accuracy - 0.89

**Context**

According to the World Health Organization (WHO) stroke is the 2nd leading cause of death globally, responsible for approximately 11% of total deaths. 
This dataset is used to predict whether a patient is likely to get stroke based on the input parameters like gender, age, various diseases, and smoking status. Each row in the data provides relavant information about the patient.

**Attribute Information**

- 1) id: unique identifier
- 2) gender: "Male", "Female" or "Other"
- 3) age: age of the patient
- 4) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
- 5) heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease
- 6) ever_married: "No" or "Yes"
- 7) work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"
- 8) Residence_type: "Rural" or "Urban"
- 9) avg_glucose_level: average glucose level in blood
- 10) bmi: body mass index
- 11) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*
- 12) stroke: 1 if the patient had a stroke or 0 if not

*Note: "Unknown" in smoking_status means that the information is unavailable for this patient



---
### Contents(index)
```
Step 1. Data Load & EDA
Step 2. Feature Engineering
     2-a. Binary Features
     2-b. Continuous Features
     2-c. Categorical Features
Step 3. Train / Test set Split & Upsampling
Step 4. Modeling & Prediction
```

### Step 1. Data Load & EDA

In [ ]:
import pandas as pd

In [ ]:
ls

In [ ]:
df = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.nunique()

In [ ]:
df.gender.unique()

In [ ]:
df.work_type.unique()

In [ ]:
df.smoking_status.unique()

In [ ]:
df.bmi

In [ ]:
df.avg_glucose_level

---
- ID : delete
- Age : transform to Category ( 20s, 30s, 40s ...)
- Gender : transfrom by One-hot Encoding
- work_type : transfrom by One-hot Encoding
- smoking_status : transfrom by One-hot Encoding
- bmi : transform to Category ( 20-29, 30-39 ...)
- avg_glucose_level : transform to Category ( 0-50, 51-100 ...)
- Others : binary

---
It's sparse table, so we use RandomForest and GradientBoosting Algorithm instead of KNN

In [ ]:
df

### Step 2. Feature Engineering

In [ ]:
df.gender.value_counts()

In [ ]:
df[df['gender'] == 'Other']

In [ ]:
df[df['gender'] == 'Other'].index

In [ ]:
df.drop(df[df['gender'] == 'Other'].index, inplace=True)

In [ ]:
df

### 2-a. Feature Engineering - Binary Features

1. gender : 0 or 1

In [ ]:
df['gender'] = df['gender'].apply(lambda x : 0 if x == 'Female' else 1)

2. ever_married : 0 or 1

In [ ]:
df['ever_married'].apply(lambda x : 0 if x == 'No' else 1).unique()

In [ ]:
df['ever_married'] = df['ever_married'].apply(lambda x : 0 if x == 'No' else 1)

3. Residence_type

In [ ]:
df['Residence_type'].value_counts()

In [ ]:
df['Residence_type'] = df['Residence_type'].apply(lambda x : 0 if x == 'Rural' else 1)

In [ ]:
df

### 2-b. Feature Engineering - Continuous Features

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings("ignore")

1. age

In [ ]:
sns.distplot(df['age']);

In [ ]:
def age_classifier(age):
    if age < 20 :
        return 'age_under 19'
    elif age < 40 :
        return 'age_20 to 39'
    elif age < 60:
        return 'age_40 to 59'
    else:
        return 'age_over 60'

In [ ]:
df['age'].apply(lambda x : age_classifier(x))

In [ ]:
df['age'] = df['age'].apply(lambda x : age_classifier(x))

In [ ]:
df['age'].unique()

In [ ]:
df['age'].value_counts()

2. avg_glucose_level

In [ ]:
sns.distplot(df['avg_glucose_level']);

In [ ]:
df['avg_glucose_level'].describe()

___
let's split by 25%, 50%, 75% line

In [ ]:
def glucose_level_classifier(level):
    if level < 77:
        return 'gl_under 25%'
    elif level < 91:
        return 'gl_26% to 50%'
    elif level < 114:
        return 'gl_50% to 75%'
    else:
        return 'gl_over 75%'

In [ ]:
df['avg_glucose_level'].apply(lambda x : glucose_level_classifier(x))

In [ ]:
# let's check 'is properly distributed?'
df['avg_glucose_level'].apply(lambda x : glucose_level_classifier(x)).value_counts()

In [ ]:
df['avg_glucose_level'] = df['avg_glucose_level'].apply(lambda x : glucose_level_classifier(x))

3. bmi

In [ ]:
df['bmi'].describe()

---
Let's classify 'bmi' the same as 'avg_glucose_level' (distribution %)

In [ ]:
def bmi_classifier(figure):
    if figure < 23:
        return 'bmi_under 25%'
    elif figure < 28:
        return 'bmi_26% to 50%'
    elif figure < 33:
        return 'bmi_50% to 75%'
    else:
        return 'bmi_over 75%'

---
!! But 'bmi' has 101 null values

In [ ]:
df.info()

In [ ]:
df[df.bmi.isnull()]

We will fill null values with mean of bmi, because 50% line is similar with mean.

In [ ]:
sns.distplot(df['bmi']);

In [ ]:
df['bmi'].fillna(28, inplace=True)

In [ ]:
df.bmi.isnull().sum()

Filled well

In [ ]:
df['bmi'].apply(lambda x : bmi_classifier(x)).value_counts()

In [ ]:
df['bmi'] = df['bmi'].apply(lambda x : bmi_classifier(x))

In [ ]:
df

### 2-c. Feature Engineering - Categorical Features

In This part, We try One-Hot Encoding to All Categorical Features including we tranformed before(age, avg_glucose_level, bmi)

In [ ]:
df.info()

In [ ]:
columns = df.columns

In [ ]:
columns

In [ ]:
df[columns[0]].dtype

In [ ]:
num_cols = []
cat_cols = []
for col in columns:
    if df[col].dtype == int:
        num_cols.append(col)
    else:
        cat_cols.append(col)

In [ ]:
print('numeric columns : {}'.format(num_cols))
print('categorical columns : {}'.format(cat_cols))

1. age

In [ ]:
from sklearn.preprocessing import LabelBinarizer 

lb = LabelBinarizer()
X_encoded = lb.fit_transform(df['age']) 
X_encoded

In [ ]:
df['age']

---
- colums : [age_20 to 39 ,age_40 to 59, age_over 60, age_under 19]

In [ ]:
df['age'].unique()

In [ ]:
pd.DataFrame(X_encoded, columns=['age_20 to 39' ,'age_40 to 59', 'age_over 60', 'age_under 19'])

In [ ]:
age_df = pd.DataFrame(X_encoded, columns=['age_20 to 39' ,'age_40 to 59', 'age_over 60', 'age_under 19'])

In [ ]:
cat_cols

2. work_type

In [ ]:
one_hot_encoded = lb.fit_transform(df['work_type'])

In [ ]:
one_hot_encoded[-20:]

In [ ]:
df['work_type'].tail(20)

In [ ]:
df['work_type'].unique()

---
mapping
- colums : ['Govt_job','Never_worked', 'Private', 'Self-employed', 'Children']

In [ ]:
pd.DataFrame(one_hot_encoded, columns=['Govt_job','Never_worked', 'Private', 'Self-employed', 'Children'])

In [ ]:
work_type_df = pd.DataFrame(one_hot_encoded, columns=['Govt_job','Never_worked', 'Private', 'Self-employed', 'Children'])

3. avg_glucose_level 

In [ ]:
df['avg_glucose_level']

In [ ]:
one_hot_encoded = lb.fit_transform(df['avg_glucose_level'])

In [ ]:
one_hot_encoded

In [ ]:
df['avg_glucose_level'].unique()

mapping
- columns : ['gl_26% to 50%', 'gl_50% to 75%', 'gl_over 75%', 'gl_under 25%']

In [ ]:
pd.DataFrame(one_hot_encoded, columns=['gl_26% to 50%', 'gl_50% to 75%', 'gl_over 75%', 'gl_under 25%'])

In [ ]:
agl_df = pd.DataFrame(one_hot_encoded, columns=['gl_26% to 50%', 'gl_50% to 75%', 'gl_over 75%', 'gl_under 25%'])

4. bmi

In [ ]:
df['bmi']

In [ ]:
one_hot_encoded = lb.fit_transform(df['bmi'])
one_hot_encoded

In [ ]:
df['bmi'].unique()

---
mapping

- columns : ['bmi_26% to 50%', 'bmi_50% to 75%', 'bmi_over 75%', 'bmi_under 25%']

In [ ]:
pd.DataFrame(one_hot_encoded, columns=['bmi_26% to 50%', 'bmi_50% to 75%', 'bmi_over 75%', 'bmi_under 25%'])

In [ ]:
bmi_df = pd.DataFrame(one_hot_encoded, columns=['bmi_26% to 50%', 'bmi_50% to 75%', 'bmi_over 75%', 'bmi_under 25%'])

In [ ]:
df['smoking_status']

In [ ]:
df['smoking_status'].unique()

In [ ]:
lb.fit_transform(df['smoking_status'])

In [ ]:
one_hot_encoded = lb.fit_transform(df['smoking_status'])

mapping
- columns : ['Unknown','formerly smoked','never smoked','smokes']

In [ ]:
pd.DataFrame(one_hot_encoded, columns=['Unknown','formerly smoked','never smoked','smokes'])

In [ ]:
smoked_df = pd.DataFrame(one_hot_encoded, columns=['Unknown','formerly smoked','never smoked','smokes'])

In [ ]:
cat_cols.append('age')

In [ ]:
cat_cols

finally! merge!

In [ ]:
df

In [ ]:
df.drop(cat_cols,axis=1,inplace=True)
df

In [ ]:
# delete meaningless column (for machie learning)
df.drop('id', axis=1, inplace=True)

In [ ]:
df

In [ ]:
cat_cols

In [ ]:
df.reset_index().iloc[:,1:]

In [ ]:
df = df.reset_index().iloc[:,1:]

In [ ]:
df

In [ ]:
pd.concat([df,age_df,agl_df,work_type_df,bmi_df,smoked_df], axis=1)

In [ ]:
pd.concat([df,age_df,agl_df,work_type_df,bmi_df,smoked_df], axis=1).isnull().sum()

In [ ]:
final_df = pd.concat([df,age_df,agl_df,work_type_df,bmi_df,smoked_df], axis=1)

### Step 3. Train, Test set split & Upsampling

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = final_df.drop('stroke', axis=1)
y = final_df['stroke']

In [ ]:
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=111)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
plt.bar(x = y_train.value_counts().index, height = y_train.value_counts().values);

It's imbalanced data, we have to upsample '1' in **training set-y**

In [ ]:
from sklearn.utils import resample

In [ ]:
train_df = pd.concat([X_train,y_train], axis=1)

In [ ]:
train_df

In [ ]:
train_0 = train_df[train_df['stroke']==0]
train_1 = train_df[train_df['stroke']==1]


In [ ]:
train_0.shape, train_1.shape

In [ ]:
upsampled_train_1 = resample(train_1,
                             replace=True,
                             n_samples=3893,
                             random_state=123
                            )
upsampled_train_1.shape

In [ ]:
upsampled_train = pd.concat([train_0, upsampled_train_1])

In [ ]:
upsampled_train['stroke'].value_counts()

In [ ]:
X_train = upsampled_train.drop('stroke',axis=1)
y_train = upsampled_train['stroke']

### Step 4. Modeling & Prediction

1. RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
print('Train Accuracy : {:.2f}'.format(rfc.score(X_train, y_train)))
print('Test Accuracy : {:.2f}'.format(rfc.score(X_test, y_test)))

2. GradientBoostingClassifier

In [ ]:
best_params = {}
score = 0
from sklearn.ensemble import GradientBoostingClassifier
for i in range(1,8):
    for j in [50, 100, 150, 200, 250, 300, 350, 400]:
        gbc = GradientBoostingClassifier(max_depth=i,
                                         n_estimators=j
                                        )
        gbc.fit(X_train, y_train)

        print('max_depth : {}'.format(i))
        print('n_estimators : {}'.format(j))
        print('Train Score : {}'.format(gbc.score(X_train,y_train)))
        print('Train Score : {}'.format(gbc.score(X_test,y_test)))
        print('----------------------------------------------------')
        if gbc.score(X_test,y_test) > score:
            score = gbc.score(X_test, y_test)
            best_params['max_depth'] = i
            best_params['n_estimators'] = j

In [ ]:
best_params

In [ ]:
score